In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content

In [ ]:
get_completion("What is 1+1?")

In [ ]:
customer_message = """
Le soleil brillait doucement à travers les feuilles des arbres, projetant des ombres dansantes sur le sol. Les oiseaux chantaient joyeusement, créant une mélodie apaisante qui accompagnait le bruissement léger du vent. Au loin, on pouvait entendre le murmure d’un ruisseau, apportant une sensation de fraîcheur à cette journée paisible. Les enfants jouaient dans le jardin, leurs rires résonnant comme une symphonie de bonheur. C’était une de ces journées parfaites où le temps semblait s’arrêter, offrant un moment de sérénité rare dans la vie trépidante de tous les jours.
"""

text_style = "Respectful Canadian English"

In [ ]:
prompt = f""" Translate the text \ that is delimited by triple backticks into a style that is {style}.
Text: ```{customer_message}```
"""

In [ ]:
response = get_completion(prompt)
response

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI(temperature=0.0)

In [ ]:
template_string = """
Translate the following text into {style}:
```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template=template_string)
print(prompt_template.input_variables)

In [ ]:
customer_messages = prompt_template.format_messages(style=text_style, text=customer_message)
print(customer_messages)
print(customer_messages[0])

In [ ]:
from langchain.schema import HumanMessage

response = chat(customer_messages)
english_message = response.content
print(response.content)

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
rating_schema = ResponseSchema(name="rating", description="A rating of the grammar of the english")
respect_schema = ResponseSchema(name="respect", description="A rating of how respectful the sentence was in tone")

response_schemas = [rating_schema, respect_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
rate_english = """
Rate the respectfulness and fluidity of this English:

rating_english: How good the english was
rating_respect: How respectful the english was

text: {text}

{format_instructions}
"""

print(english_message)
rate_eng_prompt_template = ChatPromptTemplate.from_template(template=rate_english)
rate_english_template = rate_eng_prompt_template.format_messages(text=english_message, format_instructions=format_instructions)

In [ ]:
response = chat(rate_english_template)
print(response.content)

In [ ]:
response_dict = output_parser.parse(response.content)
type(response_dict)